# Part 1 Prompt

```
--- Day 7: Amplification Circuit ---
Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.
```

```
    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
```

```
The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.
```

```
For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

 - Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
 - Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
 - Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.
 - Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
 - Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.
```

Pulling code from day 5

In [1]:
def get_param_mode(param_modes_str, param_num):
    # param_num is 1, 2, or 3
    if param_num <= len(param_modes_str):
        mode_code = param_modes_str[-param_num]
        if mode_code == "0":
            return "position"
        else:
            return "immediate"
    else:
        # if it's missing from the instruction, it's always "position"
        return "position"

In [2]:
def get_index_or_value(number, position_mode, program_codes):
    if position_mode == "immediate":
        return number
    else:
        return program_codes[number]

In [3]:
def get_value_for_input(param_modes, param_number, program_codes, current_position):
    """ Now that we have to do this for the input of more than 3 opcodes, making a
    function makes more sense
    """
    input_mode = get_param_mode(param_modes, param_number)
    input_number = program_codes[current_position + param_number]
    actual_value = get_index_or_value(input_number, input_mode, program_codes)
    return actual_value

In [4]:
def run_list_program(program_codes):
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,4,5,6,7,8]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            user_input = int(input("Enter a number: "))
            destination_index = program_codes[current_position+1]
            program_codes[destination_index] = user_input
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
            print(input_1)
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            destination_index = program_codes[current_position + 3]
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4

        else:
            print("Error, encountered opcode", opcode)
            break

In [5]:
example1_input = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [6]:
run_list_program(example1_input)

Enter a number:  3
Enter a number:  0


3


In [7]:
run_list_program(example1_input)

Enter a number:  1
Enter a number:  3


31


In [8]:
run_list_program(example1_input)

Enter a number:  2
Enter a number:  31


312


In [9]:
run_list_program(example1_input)

Enter a number:  4
Enter a number:  312


3124


In [10]:
run_list_program(example1_input)

Enter a number:  0
Enter a number:  3124


31240


...I have entered the example code but have no idea if that was the expected output...

In [11]:
example2_input_str = "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"

In [12]:
run_list_program([int(x) for x in example2_input_str.split(",")])

Enter a number:  4
Enter a number:  0


4


In [13]:
run_list_program([int(x) for x in example2_input_str.split(",")])

Enter a number:  3
Enter a number:  4


43


In [14]:
run_list_program([int(x) for x in example2_input_str.split(",")])

Enter a number:  2
Enter a number:  43


432


In [15]:
run_list_program([int(x) for x in example2_input_str.split(",")])

Enter a number:  1
Enter a number:  432


4321


In [16]:
run_list_program([int(x) for x in example2_input_str.split(",")])

Enter a number:  0
Enter a number:  4321


43210


Ok, so it seems like I need to change the functionality so it isn't actually asking for user input, it's getting this list of numbers from somewhere else

In [24]:
phase_setting_sequence = [4,3,2,1,0]

In [48]:
def run_list_program(program_codes, phase_setting, previous_output):
    
    # amplifier settings
    phase_setting_time = True
    
    # loop through codes in 4-code "programs"
    current_position = 0
    while True:
        opcode_and_param_modes = str(program_codes[current_position])
        # last two digits is opcode
        opcode = int(opcode_and_param_modes[-2:])

        # made the decision to indent as little as possible, short-circuit instead
        if opcode == 99:
            break
        
        param_modes = opcode_and_param_modes[:-2]
        
        # these opcodes have a first parameter
        if opcode in [1,2,4,5,6,7,8]:
            input_1 = get_value_for_input(param_modes, 1, program_codes, current_position)
        
        # these opcodes have a second parameter
        if opcode in [1,2,5,6,7,8]:
            input_2 = get_value_for_input(param_modes, 2, program_codes, current_position)

        if opcode == 1 or opcode == 2:
            """ Given two inputs and a destination index, perform some operation on 
            the two inputs and put the result in the destination index """
            destination_index = program_codes[current_position+3]

            if opcode == 1:
                output = input_1 + input_2
            elif opcode == 2:
                output = input_1 * input_2

            program_codes[destination_index] = output

            # move to the next "program", opcodes 1 and 2 have length 4
            current_position = current_position + 4
        elif opcode == 3:
            """ Here the input comes from prompting the user, then that is copied to
            the destination index """
            if (phase_setting_time):
                user_input = phase_setting
                phase_setting_time = False
#                 print("Just used phase setting", user_input)
            else:
                user_input = previous_output
#                 print("Just used previous input", user_input)
#             user_input = int(input("Enter a number: "))

           
            destination_index = program_codes[current_position+1]
            program_codes[destination_index] = user_input
            current_position = current_position + 2
        elif opcode == 4:
            """ Given one input (retrieved above), print it """
#             print(input_1)
            previous_output = input_1
            current_position = current_position + 2
        elif opcode == 5 or opcode == 6:
            """ Based on the first input, conditionally sets current_position to
            the value of the second input """
            if opcode == 5:
                should_jump = input_1 != 0
            elif opcode == 6:
                should_jump = input_1 == 0
                
            if should_jump:
                current_position = input_2
            else:
                current_position = current_position + 3
        elif opcode == 7 or opcode == 8:
            """ Based on a comparison of the first and second inputs, sets
            destination index to 1 or 0 """
            if opcode == 7:
                comparison_true = input_1 < input_2
            elif opcode == 8:
                comparison_true = input_1 == input_2
                
            destination_index = program_codes[current_position + 3]
            if comparison_true:
                program_codes[destination_index] = 1
            else:
                program_codes[destination_index] = 0
                
            current_position = current_position + 4

        else:
            print("Error, encountered opcode", opcode)
            break
    return previous_output

In [49]:
def outer_program_loop(input_str, phase_settings, previous_output):
    for phase_setting in phase_settings:
        result = run_list_program([int(x) for x in input_str.split(",")], phase_setting, previous_output)
        previous_output = result
    

In [50]:
phase_setting_sequence = [4,3,2,1,0]
example2_input_str = "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"
outer_program_loop(example2_input_str, phase_setting_sequence, 0)

4
43
432
4321
43210


In [51]:
phase_setting_sequence_3 = [0,1,2,3,4]
example3_input_str = "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0"
outer_program_loop(example3_input_str, phase_setting_sequence_3, 0)

5
54
543
5432
54321


In [52]:
def outer_program_loop(input_str, phase_settings):
    previous_output = 0
    for phase_setting in phase_settings:
        result = run_list_program([int(x) for x in input_str.split(",")], phase_setting, previous_output)
        previous_output = result
    return previous_output

In [53]:
phase_setting_sequence_4 = [1,0,4,3,2]
example4_input_str = "3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0"
outer_program_loop(example4_input_str, phase_setting_sequence_4)

6
65
652
6521
65210


65210

In [54]:
part1_input_file = open("input.txt", "r")
part1_input = part1_input_file.readline()
part1_input = part1_input.strip()
part1_input

'3,8,1001,8,10,8,105,1,0,0,21,42,67,88,105,114,195,276,357,438,99999,3,9,101,4,9,9,102,3,9,9,1001,9,2,9,102,4,9,9,4,9,99,3,9,1001,9,4,9,102,4,9,9,101,2,9,9,1002,9,5,9,1001,9,2,9,4,9,99,3,9,1001,9,4,9,1002,9,4,9,101,2,9,9,1002,9,2,9,4,9,99,3,9,101,4,9,9,102,3,9,9,1001,9,5,9,4,9,99,3,9,102,5,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,100

In [55]:
from itertools import permutations

In [56]:
possible_inputs = (0,1,2,3,4)
perm = permutations(possible_inputs)

In [57]:
max_output = 0
for p in perm:
    current_output = outer_program_loop(part1_input, p)
    if current_output > max_output:
        print("Found new max output", current_output)
        max_output = current_output

56
1212
9732
29213
146065
Found new max output 146065
56
1212
9732
48660
145997
56
1212
3653
29260
146300
Found new max output 146300
56
1212
3653
18265
146156
56
1212
6060
48516
145565
56
1212
6060
18197
145612
56
484
9772
29333
146665
Found new max output 146665
56
484
9772
48860
146597
56
484
1469
29472
147360
Found new max output 147360
56
484
1469
7345
146992
56
484
2420
48492
145493
56
484
2420
7277
145632
56
185
3792
30372
151860
Found new max output 151860
56
185
3792
18960
151716
56
185
1516
30412
152060
Found new max output 152060
56
185
1516
7580
151692
56
185
925
18592
148772
56
185
925
7436
148812
56
280
5692
45572
136733
56
280
5692
17093
136780
56
280
2276
45612
136853
56
280
2276
6845
136992
56
280
857
17232
137892
56
280
857
6892
137932
92
1160
9316
27965
139825
92
1160
9316
46580
139757
92
1160
3497
28012
140060
92
1160
3497
17485
139916
92
1160
5800
46436
139325
92
1160
5800
17417
139372
92
772
9320
27977
139885
92
772
9320
46600
139817
92
772
2333
28052
140260
92
77

In [58]:
max_output

212460

# Part 2 Prompt
```
--- Part Two ---
It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need. The Elves quickly talk you through rewiring the amplifiers into a feedback loop:

      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
```

```
Most of the amplifiers are connected as they were before; amplifier A's output is connected to amplifier B's input, and so on. However, the output from amplifier E is now connected into amplifier A's input. This creates the feedback loop: the signal will be sent through the amplifiers many times.

In feedback loop mode, the amplifiers need totally different phase settings: integers from 5 to 9, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.

Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.

All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a 0 signal is sent to amplifier A's input exactly once.

Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier E is sent to the thrusters. Your job is to find the largest output signal that can be sent to the thrusters using the new phase settings and feedback loop arrangement.
```

```
Here are some example programs:

Max thruster signal 139629729 (from phase setting sequence 9,8,7,6,5):

3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5


Max thruster signal 18216 (from phase setting sequence 9,7,8,5,6):

3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10

Try every combination of the new phase settings on the amplifier feedback loop. What is the highest signal that can be sent to the thrusters?
```

In [59]:
def outer_program_loop(input_str, phase_settings):
    previous_output = 0
    while True:
        loop_output = previous_output
        for phase_setting in phase_settings:
            result = run_list_program([int(x) for x in input_str.split(",")], phase_setting, previous_output)
            previous_output = result
        if (loop_output == previous_output):
            break
    return previous_output

^ that approach just creates an infinite loop.  How does it know when to stop?

I went ahead and looked at the subreddit, which seems to say that amp E getting code 99 is the "real" stopping point

In [167]:
class Amplifier:
    def __init__(self, program_codes):
        self.current_position = 0
        self.program_codes = program_codes
        
    def run_list_program(self, current_input):
        # continue running as long as no IO operation needs to happen
        while True:
            opcode_and_param_modes = str(self.program_codes[self.current_position])
            # last two digits is opcode
            opcode = int(opcode_and_param_modes[-2:])
            
            # made the decision to indent as little as possible, short-circuit instead
            if opcode == 99:
                return current_input, "Done"
            
            param_modes = opcode_and_param_modes[:-2]
            
            # these opcodes have a first parameter
            if opcode in [1,2,4,5,6,7,8]:
                input_1 = get_value_for_input(param_modes, 1, self.program_codes, self.current_position)

            # these opcodes have a second parameter
            if opcode in [1,2,5,6,7,8]:
                input_2 = get_value_for_input(param_modes, 2, self.program_codes, self.current_position)

            if opcode == 1 or opcode == 2:
                """ Given two inputs and a destination index, perform some operation on 
                the two inputs and put the result in the destination index """
                destination_index = self.program_codes[self.current_position+3]

                if opcode == 1:
                    output = input_1 + input_2
                elif opcode == 2:
                    output = input_1 * input_2

                self.program_codes[destination_index] = output

                # move to the next "program", opcodes 1 and 2 have length 4
                self.current_position = self.current_position + 4
                
            elif opcode == 3:
                """ Here the input comes from the function argument, then that is copied to
                the destination index """
                if (current_input != None):
#                     print("Using input", current_input)
                    destination_index = self.program_codes[self.current_position+1]
                    self.program_codes[destination_index] = current_input
                    current_input = None
                    self.current_position = self.current_position + 2
                else:
                    return None, "Awaiting input"
                
            elif opcode == 4:
                """ Given one input (retrieved above), print it """
                self.current_position = self.current_position + 2
                return input_1, "Output ready"
            
            elif opcode == 5 or opcode == 6:
                """ Based on the first input, conditionally sets current_position to
                the value of the second input """
                if opcode == 5:
                    should_jump = input_1 != 0
                elif opcode == 6:
                    should_jump = input_1 == 0

                if should_jump:
                    self.current_position = input_2
                else:
                    self.current_position = self.current_position + 3
            elif opcode == 7 or opcode == 8:
                """ Based on a comparison of the first and second inputs, sets
                destination index to 1 or 0 """
                if opcode == 7:
                    comparison_true = input_1 < input_2
                elif opcode == 8:
                    comparison_true = input_1 == input_2

                destination_index = self.program_codes[self.current_position + 3]
                if comparison_true:
                    self.program_codes[destination_index] = 1
                else:
                    self.program_codes[destination_index] = 0

                self.current_position = self.current_position + 4

            else:
                print("Error, encountered opcode", opcode)
                break
            

Let's see if the Part 1 structure still works here

No, something is wrong.  It should be returning 4 right now...because 0 is false-y.  Gotta check for None explicitly

In [92]:
example2_input_str = "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"
A = Amplifier([int(x) for x in example2_input_str.split(",")])
B = Amplifier([int(x) for x in example2_input_str.split(",")])
C = Amplifier([int(x) for x in example2_input_str.split(",")])
D = Amplifier([int(x) for x in example2_input_str.split(",")])
E = Amplifier([int(x) for x in example2_input_str.split(",")])

In [93]:
A.run_list_program(4)

Using input 4


(None, 'Awaiting input')

In [94]:
A.run_list_program(0)

Using input 0


(4, 'Output ready')

In [95]:
B.run_list_program(3)

Using input 3


(None, 'Awaiting input')

In [96]:
B.run_list_program(4)

Using input 4


(43, 'Output ready')

In [97]:
C.run_list_program(2)

Using input 2


(None, 'Awaiting input')

In [98]:
C.run_list_program(43)

Using input 43


(432, 'Output ready')

In [99]:
D.run_list_program(1)

Using input 1


(None, 'Awaiting input')

In [100]:
D.run_list_program(432)

Using input 432


(4321, 'Output ready')

In [101]:
E.run_list_program(0)

Using input 0


(None, 'Awaiting input')

In [102]:
E.run_list_program(4321)

Using input 4321


(43210, 'Output ready')

Ok, that looks fine.  Now let's walk through the example from Part 2

In [103]:
example4_input_str = "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
A = Amplifier([int(x) for x in example4_input_str.split(",")])
B = Amplifier([int(x) for x in example4_input_str.split(",")])
C = Amplifier([int(x) for x in example4_input_str.split(",")])
D = Amplifier([int(x) for x in example4_input_str.split(",")])
E = Amplifier([int(x) for x in example4_input_str.split(",")])

In [104]:
A.run_list_program(9)

Using input 9


(None, 'Awaiting input')

In [105]:
A.run_list_program(0)

Using input 0


(5, 'Output ready')

In [106]:
B.run_list_program(8)

Using input 8


(None, 'Awaiting input')

In [107]:
B.run_list_program(5)

Using input 5


(14, 'Output ready')

In [108]:
C.run_list_program(7)

Using input 7


(None, 'Awaiting input')

In [109]:
C.run_list_program(14)

Using input 14


(31, 'Output ready')

In [110]:
D.run_list_program(6)

Using input 6


(None, 'Awaiting input')

In [111]:
D.run_list_program(31)

Using input 31


(64, 'Output ready')

In [112]:
E.run_list_program(5)

Using input 5


(None, 'Awaiting input')

In [113]:
E.run_list_program(64)

Using input 64


(129, 'Output ready')

In [114]:
A.run_list_program(129)

Using input 129


(263, 'Output ready')

In [115]:
B.run_list_program(263)

Using input 263


(530, 'Output ready')

In [116]:
C.run_list_program(530)

Using input 530


(1063, 'Output ready')

In [117]:
D.run_list_program(1063)

Using input 1063


(2128, 'Output ready')

In [118]:
E.run_list_program(2128)

Using input 2128


(4257, 'Output ready')

In [119]:
A.run_list_program(4257)

Using input 4257


(8519, 'Output ready')

In [120]:
B.run_list_program(8519)

Using input 8519


(17042, 'Output ready')

In [121]:
C.run_list_program(17042)

Using input 17042


(34087, 'Output ready')

In [122]:
D.run_list_program(34087)

Using input 34087


(68176, 'Output ready')

In [123]:
E.run_list_program(68176)

Using input 68176


(136353, 'Output ready')

In [124]:
A.run_list_program(136353)

Using input 136353


(272711, 'Output ready')

In [125]:
B.run_list_program(272711)

Using input 272711


(545426, 'Output ready')

In [126]:
C.run_list_program(545426)

Using input 545426


(1090855, 'Output ready')

In [127]:
D.run_list_program(1090855)

Using input 1090855


(2181712, 'Output ready')

In [128]:
E.run_list_program(2181712)

Using input 2181712


(4363425, 'Output ready')

In [129]:
A.run_list_program(4363425)

Using input 4363425


(8726855, 'Output ready')

In [130]:
B.run_list_program(8726855)

Using input 8726855


(17453714, 'Output ready')

In [131]:
C.run_list_program(17453714)

Using input 17453714


(34907431, 'Output ready')

In [132]:
D.run_list_program(4907431)

Using input 4907431


(9814864, 'Output ready')

In [133]:
E.run_list_program(9814864)

Using input 9814864


(19629729, 'Output ready')

In [134]:
A.run_list_program(19629729)

(19629729, 'Done')

Well that was tedious...and it didn't work

Or...let's see what happens if we get to the end, maybe it really must be E with code 99

In [135]:
B.run_list_program(19629729)

(19629729, 'Done')

In [136]:
C.run_list_program(19629729)

(19629729, 'Done')

In [137]:
D.run_list_program(19629729)

(19629729, 'Done')

In [138]:
E.run_list_program(19629729)

(19629729, 'Done')

Nope, that didn't work

Let's see just in case, if I automate it instead of doing it by hand, if that fixes the problem

In [139]:
example4_input_str = "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
A = Amplifier([int(x) for x in example4_input_str.split(",")])
B = Amplifier([int(x) for x in example4_input_str.split(",")])
C = Amplifier([int(x) for x in example4_input_str.split(",")])
D = Amplifier([int(x) for x in example4_input_str.split(",")])
E = Amplifier([int(x) for x in example4_input_str.split(",")])

In [140]:
A.run_list_program(9)

Using input 9


(None, 'Awaiting input')

In [141]:
B.run_list_program(8)

Using input 8


(None, 'Awaiting input')

In [142]:
C.run_list_program(7)

Using input 7


(None, 'Awaiting input')

In [143]:
D.run_list_program(6)

Using input 6


(None, 'Awaiting input')

In [144]:
E.run_list_program(5)

Using input 5


(None, 'Awaiting input')

In [146]:
current_amp = A
value = 0
while True:
    value, status = current_amp.run_list_program(value)
    if status == "Done":
        break
        
    if current_amp == A:
        current_amp = B
    elif current_amp == B:
        current_amp = C
    elif current_amp == C:
        current_amp = D
    elif current_amp == D:
        current_amp = E
    else:
        current_amp = A

Using input 0
Using input 5
Using input 14
Using input 31
Using input 64
Using input 129
Using input 263
Using input 530
Using input 1063
Using input 2128
Using input 4257
Using input 8519
Using input 17042
Using input 34087
Using input 68176
Using input 136353
Using input 272711
Using input 545426
Using input 1090855
Using input 2181712
Using input 4363425
Using input 8726855
Using input 17453714
Using input 34907431
Using input 69814864


In [147]:
value

139629729

In [148]:
expecting = 139629729

In [149]:
value == expecting

True

Ok cool, I must have done something wrong in looping by hand.  Let's test the other example

In [156]:
example5_input_str = "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"
A = Amplifier([int(x) for x in example5_input_str.split(",")])
B = Amplifier([int(x) for x in example5_input_str.split(",")])
C = Amplifier([int(x) for x in example5_input_str.split(",")])
D = Amplifier([int(x) for x in example5_input_str.split(",")])
E = Amplifier([int(x) for x in example5_input_str.split(",")])

In [157]:
phase_setting_sequence_5 = [9,7,8,5,6]

In [158]:
all_amps = [A, B, C, D, E]

In [159]:
for index, amp in enumerate(all_amps):
    amp.run_list_program(phase_setting_sequence_5[index])

Using input 9
Using input 7
Using input 8
Using input 5
Using input 6


In [160]:
current_amp_index = 0
value = 0
while True:
    current_amp = all_amps[current_amp_index%5]
    value, status = current_amp.run_list_program(value)
    if status == "Done":
        break
        
    current_amp_index += 1

Using input 0
Using input 4
Using input 6
Using input 9
Using input 18
Using input 19
Using input 22
Using input 23
Using input 25
Using input 29
Using input 58
Using input 60
Using input 120
Using input 121
Using input 124
Using input 128
Using input 129
Using input 133
Using input 266
Using input 268
Using input 271
Using input 542
Using input 545
Using input 549
Using input 550
Using input 552
Using input 556
Using input 558
Using input 561
Using input 1122
Using input 1123
Using input 1126
Using input 1127
Using input 1129
Using input 1133
Using input 2266
Using input 2268
Using input 4536
Using input 4537
Using input 4540
Using input 4544
Using input 4545
Using input 4549
Using input 9098
Using input 9100
Using input 9103
Using input 18206
Using input 18209
Using input 18213
Using input 18214


In [161]:
value

18216

In [162]:
expected = 18216

In [163]:
value == expected

True

Ok, I understand how it processes input now, just need to iterate over the permutations for the full input again

In [164]:
def test_phase_settings(input_str, phase_settings):
    
    A = Amplifier([int(x) for x in input_str.split(",")])
    B = Amplifier([int(x) for x in input_str.split(",")])
    C = Amplifier([int(x) for x in input_str.split(",")])
    D = Amplifier([int(x) for x in input_str.split(",")])
    E = Amplifier([int(x) for x in input_str.split(",")])
    
    all_amps = [A, B, C, D, E]
    
    for index, amp in enumerate(all_amps):
        amp.run_list_program(phase_settings[index])
    
    current_amp_index = 0
    value = 0
    while True:
        current_amp = all_amps[current_amp_index%5]
        value, status = current_amp.run_list_program(value)
        if status == "Done":
            break

        current_amp_index += 1

    return value

In [165]:
test_phase_settings(example5_input_str, phase_setting_sequence_5)

Using input 9
Using input 7
Using input 8
Using input 5
Using input 6
Using input 0
Using input 4
Using input 6
Using input 9
Using input 18
Using input 19
Using input 22
Using input 23
Using input 25
Using input 29
Using input 58
Using input 60
Using input 120
Using input 121
Using input 124
Using input 128
Using input 129
Using input 133
Using input 266
Using input 268
Using input 271
Using input 542
Using input 545
Using input 549
Using input 550
Using input 552
Using input 556
Using input 558
Using input 561
Using input 1122
Using input 1123
Using input 1126
Using input 1127
Using input 1129
Using input 1133
Using input 2266
Using input 2268
Using input 4536
Using input 4537
Using input 4540
Using input 4544
Using input 4545
Using input 4549
Using input 9098
Using input 9100
Using input 9103
Using input 18206
Using input 18209
Using input 18213
Using input 18214


18216

In [166]:
possible_inputs = (5, 6, 7, 8, 9)
perm = permutations(possible_inputs)

In [168]:
max_output = 0
for p in perm:
    current_output = test_phase_settings(part1_input, p)
    if current_output > max_output:
        print("Found new max output", current_output)
        max_output = current_output

Found new max output 18429706
Found new max output 19552258
Found new max output 19556385
Found new max output 19556444
Found new max output 21797730
Found new max output 21801857
Found new max output 21810049
Found new max output 21832418
Found new max output 21836545
Found new max output 21844737
